In [34]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os
from importlib import reload
from torch.utils.data import DataLoader,Dataset

数据处理
--

1 得到原始数据和离散，连续列

In [35]:
data_path='/media/xuweijia/DATA/代码/python_test/data/Criteo/demo_data/'
file_name='train.csv'

In [36]:
# get raw data
raw_df=pd.read_csv(os.path.join(data_path+file_name))
raw_df=raw_df.drop(["Id"],axis=1)
raw_df

,Label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,1,1.0,0,1.0,NaN,227.0,1.0,173.0,18.0,50.0,...,3486227d,e88ffc9d,c393dc22,b1252a9d,57c90cd9,NaN,bcdee96c,4d19a3eb,cb079c2d,456c12a0
1,1,4.0,1,1.0,2.0,27.0,2.0,4.0,2.0,2.0,...,07c540c4,92555263,NaN,NaN,242bb710,NaN,3a171ecb,72c78f11,NaN,NaN
2,1,0.0,806,NaN,NaN,1752.0,142.0,2.0,0.0,50.0,...,07c540c4,25c88e42,21ddcdc9,b1252a9d,a0136dd2,NaN,32c7478e,8fc66e78,001f3601,f37f3967
3,0,2.0,-1,42.0,14.0,302.0,38.0,25.0,38.0,90.0,...,e5ba7672,5aed7436,21ddcdc9,b1252a9d,c3abeb21,NaN,423fab69,1793a828,e8b83407,5cef228f
4,1,0.0,57,2.0,1.0,2891.0,2.0,35.0,1.0,137.0,...,e5ba7672,642f2610,1d1eb838,b1252a9d,1640d50b,ad3062eb,423fab69,45ab94c8,2bf691b1,c84c4aec
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,0,NaN,8,1.0,1.0,43.0,NaN,0.0,1.0,1.0,...,1e88c74f,fc35e8fe,NaN,NaN,a02708ad,c9d4222a,c3dc6cef,502f2493,NaN,NaN
1595,0,8.0,2,20.0,8.0,36.0,9.0,8.0,10.0,8.0,...,e5ba7672,5aed7436,21ddcdc9,b1252a9d,eea796be,NaN,3a171ecb,1793a828,e8b83407,5cef228f
1596,0,0.0,1,2.0,12.0,4877.0,140.0,13.0,34.0,136.0,...,e5ba7672,2b0a9d11,NaN,NaN,7453e535,NaN,dbb486d7,906e72ec,NaN,NaN
1597,0,NaN,2,NaN,1.0,1972.0,NaN,0.0,1.0,14.0,...,e5ba7672,817481a8,NaN,NaN,e4244d7f,c9d4222a,c7dc6720,60efe6e6,NaN,NaN


In [37]:
# 分别找出连续列/离散列
def col_type(df):
    dis_col=[]
    con_col=[]
    columns=df.columns.tolist()
    for c in columns:
        if df[c].dtype=='int64' or df[c].dtype=='float':
            con_col.append(c)
        else:
            dis_col.append(c)
    return dis_col,con_col
dis_col,con_col=col_type(raw_df)
con_col.remove("Label")
target="Label"

In [38]:
raw_df[con_col].values.dtype

dtype('float64')

In [39]:
# 默认是float64(double). 降低到float32. 与torch默认的兼容
raw_df[con_col]=raw_df[con_col].astype(np.float32)

In [40]:
raw_df[con_col].values.dtype

dtype('float32')

2 填充缺失值：数值型填0； 类别填空字符串，到时候也编码进去 （测试数据的缺失值用同样字符填充。相同编码）

In [41]:
null_token = '<NULL>'
raw_df[dis_col]=raw_df[dis_col].fillna(null_token)
raw_df[con_col]=raw_df[con_col].fillna(0)
raw_df

,Label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,1,1.0,0.0,1.0,0.0,227.0,1.0,173.0,18.0,50.0,...,3486227d,e88ffc9d,c393dc22,b1252a9d,57c90cd9,<NULL>,bcdee96c,4d19a3eb,cb079c2d,456c12a0
1,1,4.0,1.0,1.0,2.0,27.0,2.0,4.0,2.0,2.0,...,07c540c4,92555263,<NULL>,<NULL>,242bb710,<NULL>,3a171ecb,72c78f11,<NULL>,<NULL>
2,1,0.0,806.0,0.0,0.0,1752.0,142.0,2.0,0.0,50.0,...,07c540c4,25c88e42,21ddcdc9,b1252a9d,a0136dd2,<NULL>,32c7478e,8fc66e78,001f3601,f37f3967
3,0,2.0,-1.0,42.0,14.0,302.0,38.0,25.0,38.0,90.0,...,e5ba7672,5aed7436,21ddcdc9,b1252a9d,c3abeb21,<NULL>,423fab69,1793a828,e8b83407,5cef228f
4,1,0.0,57.0,2.0,1.0,2891.0,2.0,35.0,1.0,137.0,...,e5ba7672,642f2610,1d1eb838,b1252a9d,1640d50b,ad3062eb,423fab69,45ab94c8,2bf691b1,c84c4aec
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,0,0.0,8.0,1.0,1.0,43.0,0.0,0.0,1.0,1.0,...,1e88c74f,fc35e8fe,<NULL>,<NULL>,a02708ad,c9d4222a,c3dc6cef,502f2493,<NULL>,<NULL>
1595,0,8.0,2.0,20.0,8.0,36.0,9.0,8.0,10.0,8.0,...,e5ba7672,5aed7436,21ddcdc9,b1252a9d,eea796be,<NULL>,3a171ecb,1793a828,e8b83407,5cef228f
1596,0,0.0,1.0,2.0,12.0,4877.0,140.0,13.0,34.0,136.0,...,e5ba7672,2b0a9d11,<NULL>,<NULL>,7453e535,<NULL>,dbb486d7,906e72ec,<NULL>,<NULL>
1597,0,0.0,2.0,0.0,1.0,1972.0,0.0,0.0,1.0,14.0,...,e5ba7672,817481a8,<NULL>,<NULL>,e4244d7f,c9d4222a,c7dc6720,60efe6e6,<NULL>,<NULL>


3 可以做一些特征处理上的优化。比如数值型归一化。离散特征出现次数小于某阈值的，值都编码成\<UNK\>。这里忽略，假设已经做过了.也做过了特征选择

4 离散特征label-encode. 保存原始值到label的映射。之后根据映射后的id找对应embedding （取值10个以内的one-hot,作为新特征）
  如果想同一列加工出不同特征。可以用FeatureUnion和自定义transformer来选择列。 （如对文本列同时加工长度和tfidf两个特征）
  ColumnTransformer对同一列只能做一个操作。如果不对同一列做不同操作，就用这个就可以。

In [42]:
# %load FM_helper/LabelEncoder.py

In [43]:
# 直接当做包，引用py中函数
# labelencoding。
from FM_helper import LabelEncoder
reload(LabelEncoder)
trans,new_con_col,new_dis_col,df,raw_df2,cate_counts,cate_feature_map=LabelEncoder.labelencode_trans(raw_df,dis_col,con_col,target)
# 测试.只需要保存大transformer和最终的dis_col,con_col。 用来做转化，以及识别转换后的两类特征。 
LabelEncoder.test(raw_df,trans,new_con_col,new_dis_col,target)

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,I5,I6,I7,I8,I9,I10,I11,I12,I13,Label
0,33.0,27.0,486.0,572.0,1.0,1.0,459.0,1.0,1.0,465.0,...,227.0,1.0,173.0,18.0,50.0,1.0,7.0,1.0,0.0,1.0
1,0.0,98.0,170.0,287.0,1.0,3.0,550.0,21.0,1.0,683.0,...,27.0,2.0,4.0,2.0,2.0,1.0,1.0,0.0,2.0,1.0
2,0.0,28.0,114.0,696.0,11.0,3.0,704.0,1.0,1.0,133.0,...,1752.0,142.0,2.0,0.0,50.0,0.0,1.0,0.0,0.0,1.0
3,0.0,12.0,650.0,243.0,1.0,3.0,329.0,1.0,1.0,27.0,...,302.0,38.0,25.0,38.0,90.0,1.0,3.0,0.0,38.0,0.0
4,0.0,36.0,517.0,70.0,1.0,3.0,20.0,2.0,1.0,166.0,...,2891.0,2.0,35.0,1.0,137.0,0.0,17.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,0.0,93.0,617.0,801.0,1.0,1.0,25.0,12.0,1.0,28.0,...,43.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1595,30.0,12.0,1034.0,243.0,1.0,6.0,935.0,1.0,1.0,454.0,...,36.0,9.0,8.0,10.0,8.0,1.0,1.0,0.0,8.0,0.0
1596,30.0,113.0,676.0,7.0,1.0,6.0,185.0,14.0,1.0,485.0,...,4877.0,140.0,13.0,34.0,136.0,0.0,2.0,0.0,12.0,0.0
1597,0.0,48.0,565.0,727.0,1.0,6.0,377.0,0.0,1.0,202.0,...,1972.0,0.0,0.0,1.0,14.0,0.0,0.0,0.0,1.0,0.0


标准FM
--

公式：

$$y= b+ \sum_{i}w_ix_i  + \sum_{i}^{n}\sum_{j!=i}^{n}x_ix_j<\vec{v_i},\vec{v_j}>$$

一阶同LR. 每个连续特征对应一个$w_i$,每个离散特征one-hot之后的特征作为新特征，对应一个$w_i$

二阶交互，每个连续特征对应一个embedding。每个离散特征的每个每个特征的每个取值对应一个embedding

因此每个连续特征$x_i$,对应一个$w_i$,一个embedding，用来和其他特征交互。
   每个离散特征域，对应one-hot之后的C个特征$x_i$，对应C个$w_i$,C个embedding

但对每个样本来说，该离散特征one-hot之后，只会根据取值取到一个embedding，一个$w_i$<br/>(该离散特征对一阶的贡献，只有根据样本该离散特征取值映射到的$w_i$，对应取值$x_i$是1,其他C-1位置由于one-hot,该样本下取值$x_i$都是0,贡献是0)

因此总共需要维护（所有连续特征+所有离散特征的所有取值)个特征

假设所有连续特征和one-hot后的所有离散特征共F个,总共需要维护F个特征。可以根据特征名称，把每个特征映射到一个固定id上（位置）:<br/>
每个连续特征对应一个id                            <br/>
每个离散特征的每个取值对应一个id                    <br/>
每个id都维护一个$w_i$,一个embedding，对应该特征在W（F,1）,embedding(F,d)中的位置。<br/>
之后每个样本，都可以根据特征位置去找对应的$w_i$,embedding：       <br/>

因此在对每个样本进行映射时，需要分别得到样本每个特征的位置（id）和取值$x_i$：

位置映射：样本的所有特征都被映射到对应位置,用来找对应的$w_i$,embedding。每个连续特征对应的就是位置id。每个离散域，根据样本在该域的取值映射到对应id。n个离散域，对应n个embedding,n个$w_i$

样本取值：连续特征的取值不变（或者归一化），离散特征取值1，作为样本的$x_i$输入。n个离散域，对应的n个取值，$x_i$都是1。在one-hot后的对应位置上

### 对原始特征进行映射。得到one-hot之后的所有特征（含连续特征）到位置id的映射

In [68]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np
import os
class FeaturePosTrans(BaseEstimator, TransformerMixin):
    def __init__(self, dis_col=None, con_col= None, limit_freq = 0):
        self.dis_col=dis_col
        self.con_col=con_col
        self.limit_freq=limit_freq
        
        self.NULL = '<NULL>'
        self.UNK = '<UNK>'                                                # nlp里。低频是1，NAN是0. NAN作为padding,不参与训练且是0
                                                                          # NAN对应embedding： padding_index=0.只占位，不训练）
                                                                          # nn.Embedding(V,d,padding_idx=0
    
        self.dis_col_map=dict()                                            # 按特征，记录取值到位置id的映射  只用来存着
        self.feature_id_map=dict()                                         # 特征名到位置id的映射大表 {特征名_取值：位置id}
        self.pos=0                                                         # 位置id
        self.dis_col_count=dict()                                          # 每个离散特征的取值数目
        
        
        # 所有离散的缺失值，统一用NAN编码，之后在w，E中padding成0
        self.feature_id_map[self.NULL]=0
        self.pos+=1
        
        if (con_col!=None):
            self.feature_id_map.update(dict(zip(con_col,range(self.pos,self.pos+len(con_col))))) # 连续特征到对应位置的映射
            self.pos+=len(self.con_col)

    def fit(self, X , y = None):
        
        if (self.dis_col!=None):
            # 每个离散特征取值,映射到对应id
            for col in self.dis_col:
                valueCount=dict(X[col].value_counts())                       # 该离散特征。每个取值的出现数目
                # 是否特殊处理低频取值
                if self.limit_freq>0:
                    values=[k for k,v in valueCount.items() if k!=self.NULL and v>self.limit_freq]  # 该特征留下的取值
                    self.dis_col_map[col]=dict(zip(values,range(self.pos+1,self.pos+1+len(values))))
                    self.dis_col_map[col][self.UNK]=self.pos
                    # 组织大表。类似
                    new_values=[col+"_"+v for v in values]                    # { C1_v1：id}  
                    self.feature_id_map.update(dict(zip(new_values,range(self.pos+1,self.pos+1+len(new_values)))))
                    self.feature_id_map[col+"_"+self.UNK]=self.pos
                    self.pos+=len(new_values)+1                              # 每个特征留下：所有高频取值+UNK                                   
                else:
                    # 每个特征。分别记录映射
                    values=[k for k in valueCount.keys() if k!=self.NULL]    # 该离散特征所有取值（除缺失值） 
                    self.dis_col_map[col]=dict(zip(values,range(self.pos,self.pos+len(values))))
                    # 类似，但根据取值记在大map里
                    new_values=[col+"_"+v for v in values]                   # { C1_v1：id}  
                    self.feature_id_map.update(dict(zip(new_values,range(self.pos,self.pos+len(new_values)))))
                    self.pos+=len(new_values)
                    
                 # 每个离散特征的有效取值数目(不含NAN，含每个特征的unk)
                self.dis_col_count[col]=len(self.dis_col_map[col])                            
                                                                               
    def transform(self, X, y=None):
        # 映射：
        feature_pos=X.copy()                        # 样本每个特征对应的位置
        feature_values=X.copy()                     # 样本每个特征的取值。离散特征取值是1.
        cols=self.dis_col+self.con_col
        for col in cols:
            if col in self.dis_col:
                #values=X[col].apply(self.gen,args=(col,)).values
                values=X[col].apply(self.gen2,args=(col,)).values    # 组织形式不同。映射效果相同。用这个好些
                feature_pos[col]=values
                feature_values[col]=1.0
            else:
                feature_pos[col]=self.feature_id_map[col]            # 连续特征取值不变  。 位置是映射后的id 
        
        # 映射完的取值（包括离散特征取值1.0），也都变成float32
        feature_values=feature_values.astype(np.float32)
        
        return feature_pos,feature_values
        
    # 如果是多列。传入的x是该列对应的series. 输出的是这些列拼起来的df
    # 如果是单列，传入的x是该列的每个元素    输出的是该列对应的Series
    # 根据离散特征取值，返回对应的位置id
    def gen(self,x,col):
        if x==self.NULL:                                        # NAN统一映射到0
            return 0
        else:
            if x in self.dis_col_map[col]:
                return self.dis_col_map[col][x]                 # 按取值，映射到对应位置id
            else:
                if self.limit_freq>0:
                    return self.dis_col_map[col][self.UNK]       # 低频取值/没见过的值。映射到unqkey对应的编码 
                else:
                    return 0                                     # 没见过的值。映射到NAN==0。没有贡献

    # 用大表做映射。类似
    def gen2(self,x,col):    
        if x==self.NULL:                                         # NAN统一映射到0
            return 0
        else:
            x=col+"_"+x
            if x in self.feature_id_map:                         # 其他按取值，映射到对应位置id
                return self.feature_id_map[x]                 
            else:
                if self.limit_freq>0:                
                    return self.feature_id_map[col+"_"+self.UNK] # 低频取值/没见过的值。映射到该特征unqkey对应的编码 
                else:
                    return 0                                     # 没见过的值。映射到NAN。没有贡献
                
    def id2name(self):
        return dict(zip(self.feature_id_map.values(),self.feature_id_map.keys()))

In [69]:
#from FM_helper import Fmdata
#reload(Fmdata)
#f_trans=Fmdata.FeaturePosTrans(dis_col,con_col,10)
f_trans=FeaturePosTrans(dis_col,con_col,10)             # 出现10次以下的作为UNK
f_trans.fit(raw_df)
feature_pos,feature_values=f_trans.transform(raw_df)

In [70]:
len(f_trans.feature_id_map)  # 离散特征one-hot后，总的特征数目. NAN+con_col+all_dis

319

In [71]:
f_trans.feature_id_map

{'<NULL>': 0,
 'I1': 1,
 'I2': 2,
 'I3': 3,
 'I4': 4,
 'I5': 5,
 'I6': 6,
 'I7': 7,
 'I8': 8,
 'I9': 9,
 'I10': 10,
 'I11': 11,
 'I12': 12,
 'I13': 13,
 'C1_05db9164': 15,
 'C1_68fd1e64': 16,
 'C1_5a9ed9b0': 17,
 'C1_8cf07265': 18,
 'C1_be589b51': 19,
 'C1_5bfa8ab5': 20,
 'C1_f473b8dc': 21,
 'C1_87552397': 22,
 'C1_ae82ea21': 23,
 'C1_39af2607': 24,
 'C1_9a89b36c': 25,
 'C1_<UNK>': 14,
 'C2_38a947a1': 27,
 'C2_09e68b86': 28,
 'C2_80e26c9b': 29,
 'C2_d833535f': 30,
 'C2_4f25e98b': 31,
 'C2_287130e0': 32,
 'C2_0a519c5c': 33,
 'C2_08d6d899': 34,
 'C2_4c2bc594': 35,
 'C2_38d50e09': 36,
 'C2_207b2d81': 37,
 'C2_58e67aaf': 38,
 'C2_2c16a946': 39,
 'C2_942f9a8d': 40,
 'C2_8947f767': 41,
 'C2_421b43cd': 42,
 'C2_0468d672': 43,
 'C2_8084ee93': 44,
 'C2_78ccd99e': 45,
 'C2_1cfdf714': 46,
 'C2_68b3edbf': 47,
 'C2_e112a9de': 48,
 'C2_95e2d337': 49,
 'C2_e5fb1af3': 50,
 'C2_39dfaa0d': 51,
 'C2_e77e5e6e': 52,
 'C2_9819deea': 53,
 'C2_8cc9c66e': 54,
 'C2_f0cf0024': 55,
 'C2_3df44d94': 56,
 'C2_ae46a2

In [72]:
feature_pos

,Label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,1,1,2,3,4,5,6,7,8,9,...,213,240,249,257,258,0,279,284,305,309
1,1,1,2,3,4,5,6,7,8,9,...,212,219,0,0,258,0,276,284,0,0
2,1,1,2,3,4,5,6,7,8,9,...,212,219,250,257,258,0,275,284,299,309
3,0,1,2,3,4,5,6,7,8,9,...,210,220,250,257,258,0,277,287,298,309
4,1,1,2,3,4,5,6,7,8,9,...,210,219,249,257,258,270,277,290,301,311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,0,1,2,3,4,5,6,7,8,9,...,215,219,0,0,258,271,274,284,0,0
1595,0,1,2,3,4,5,6,7,8,9,...,210,220,250,257,258,0,276,287,298,309
1596,0,1,2,3,4,5,6,7,8,9,...,210,219,0,0,258,0,282,284,0,0
1597,0,1,2,3,4,5,6,7,8,9,...,210,219,0,0,258,271,280,284,0,0


In [73]:
feature_values

,Label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,1.0,1.0,0.0,1.0,0.0,227.0,1.0,173.0,18.0,50.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,4.0,1.0,1.0,2.0,27.0,2.0,4.0,2.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,0.0,806.0,0.0,0.0,1752.0,142.0,2.0,0.0,50.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0.0,2.0,-1.0,42.0,14.0,302.0,38.0,25.0,38.0,90.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,0.0,57.0,2.0,1.0,2891.0,2.0,35.0,1.0,137.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,0.0,0.0,8.0,1.0,1.0,43.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1595,0.0,8.0,2.0,20.0,8.0,36.0,9.0,8.0,10.0,8.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1596,0.0,0.0,1.0,2.0,12.0,4877.0,140.0,13.0,34.0,136.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1597,0.0,0.0,2.0,0.0,1.0,1972.0,0.0,0.0,1.0,14.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [74]:
feature_values.values.dtype

dtype('float32')

In [75]:
feature_pos=feature_pos.drop([target], axis=1)
feature_values=feature_values.drop([target], axis=1)    # 特征列去掉label

In [78]:
allcols=dis_col+con_col
feature_values[allcols].values.dtype

dtype('float32')

### 建立自己的dataset和对应的dataloader

In [79]:
torch.set_default_dtype(torch.float32)           # torch模型参数的默认数据类型是flaot32.  np/pd是默认是flaot64(double).转成一样的

In [80]:
# 建dataset: 都放np,防止loader开多进程内存泄露:https://github.com/pytorch/pytorch/issues/13246#issuecomment-893198671）
class Mydata(Dataset):
    def __init__(self,fv,fp,target,mode='train'):
        super(Mydata, self).__init__()
        self.fv=fv           # np: m,n.  每个样本的特征取值 
        self.fp=fp           #           每个样本的特征位置.如果太大以后可以放np文件名.或切成多个文件,每次只打开一个(类似drml)
        self.target=target   #           如果mode==train/valid.对应y. mode==test,没有y，对应id 
    def __len__(self):
        return len(self.fv)
    def __getitem__(self, index):
        return self.fp[index,:],self.fv[index],self.target[index]  # 提前做好了映射。当然映射也可以在这里做。

allcols=dis_col+con_col
trainDataset=Mydata(feature_values[allcols].values,feature_pos[allcols].values, raw_df[target].values)

In [81]:
# DataLoader:   valid同
trainloader = DataLoader(trainDataset,        
                    shuffle=True,             # 每个epoch全部shuffle
                    batch_size=32,
                    collate_fn=None,          # 自定义如何拼batch。默认有，一般不需要。可以用来对batch padding（根据每个batch最长的text）
                                              # 传入的是一个batch，B个tuple. 每个tuple对应Dataset传出来的n个元素。
                                              # 自己重新拼成n个元素，每个元素B行，作为loader每次迭代的返回
                    pin_memory=False,         # 页锁定内存：不可分页，占用物理内存。 可分页内存：占用虚拟内存，用时候从磁盘读入物理内存
                                                # gpu需要通过页锁定内存中，把数据复制到gpu上
                                                #  数据从cpu的可分页内存内存传到gpu时，需要先把数据复制到临时的页锁定内存，再赋值到gpu.速度更慢
                                                #  如果指定pin_memory=True, batch的数据会直接被放在cpu的页锁定内存中，传到gpu时更快。少了一步复制
                                                #  但页锁定内存会占用真实物理内存，分配过多会挤占别的程序的内存，把内存耗尽。因此内存小时不建议用
                    sampler=None,             # 定义/自定义怎么从dataset里抽取每个batch的样本.还有一个batch_sampler参数
                                              # 需要实现__iter__方法。返回针对所有样本id的迭代器，顺序按自定义的样本顺序排好
                                              # 需要实现 __len__ ， 表示loader的一次抽取完成。一般同dataset样本数。
                                              # 就不能定义shuffle了，因为策略自己实现了。shuffle只能在iter里自己做。加入随机性
                                              # 每次loader会根据__iter__(和batch_size），迭代产生该epoch的每个batch。 
                                              # 比如nlp会根据样本文本长度，将长度相近样本排一起：iter([25,3,60,0,...1])。使得每个batch长度接近
                                              # 通过在__iter_里先切好batch,再shuffle,再整合。打乱每个epoch,batch间的执行顺序
                                              # 可以设置RandomSampler每次随机采样。（可参考该实现等）                       
                    num_workers=10            # 开多进程，每个进程计算一个batch。初始化时用之前提前处理好对应的n个batch。后续batch加入新的线程
                   )                          # 初始化时会一次性建好n个进程。每个进程提前处理好要用的batch数据。（底层是 multiprocessing）

In [82]:
epoch_num=3
for epoch in range(epoch_num):
    for batch_id,x in enumerate(trainloader):   # 把数据完整轮询一遍。对应一个epoch
        # 输入模型
        break

In [83]:
x

[tensor([[15, 29, 59,  ..., 11, 12, 13],
         [15, 36, 59,  ..., 11, 12, 13],
         [16, 29, 59,  ..., 11, 12, 13],
         ...,
         [15, 26, 60,  ..., 11, 12, 13],
         [15, 26, 60,  ..., 11, 12, 13],
         [15, 40, 59,  ..., 11, 12, 13]]),
 tensor([[ 1.,  1.,  1.,  ...,  0.,  0.,  0.],
         [ 1.,  1.,  1.,  ...,  0.,  0.,  1.],
         [ 1.,  1.,  1.,  ...,  1.,  0.,  5.],
         ...,
         [ 1.,  1.,  1.,  ...,  3.,  0., 20.],
         [ 1.,  1.,  1.,  ...,  6.,  0., 31.],
         [ 1.,  1.,  1.,  ...,  2.,  0.,  2.]]),
 tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0])]

In [84]:
f_p,f_v,y=x[0],x[1],x[2]

### 原始FM模型

模型维持n个w,n个embedding，对应one-hot后的所有特征（这里还算上了缺失值对应的0参数）

二阶分数是$$\sum_{i}^{n}\sum_{j!=i}^{n}x_ix_j<\vec{v_i},\vec{v_j}>$$

直接计算是$O(n^2*k)$。每对向量内积是$k$,共$O(n^2)$个pair。最后求和

等价于embedding们对应位置22相乘后，再求和。可以先求所有embedding每个维度元素22相乘的和，最后再对所有维度求和

$$\sum_{i}^{n}\sum_{j!=i}^{n}<\vec{e_i},\vec{e_j}>= \sum_{f} \sum_{i}^{n}\sum_{j!=i}^{n}e_{if}*e_{jf} $$

embedding的每个维度元素22相乘的和： $ab+ac+bc= \frac{1}{2}((a+b+c)^2-(a^2+b^2+c^2))$

因此对固定维度，embedding的每个维度元素22相乘的和$\sum_{i}^{n}\sum_{j!=i}^{n}e_{if}*e_{jf}$ ，是$\frac{1}{2}((e_{if}+e_{jf}...)^2-(e_{if}^2+e_{if}^2+...))$

可以通过每个embedding相加（对应位置相加）的平方，减去每个embedding平方的相加，得到（1，k）的向量，作为embedding的每个维度元素22交互的结果。最终的二阶分数是这k个维度的结果相加。

也可以把这k维向量作为新的隐特征，输入后续网络。作为FM/deepFM的一个小变体

转化后，$\frac{1}{2}((e_{if}+e_{jf}...)^2-(e_{if}^2+e_{if}^2+...))$的复杂度只有$O(n)$(n个元素相加或平方后相加)。加上维度F，总的复杂度可以降低到$O(nk)$

In [85]:
class FM(nn.Module):
    def __init__(self,n_field,n_features,embed_size,dropout=0.2):
        """
        标准FM
        n_field: 原始离散特征数目
        n_features: 离散特征one-hot之后,和dense的总特征个数. 这里包含了一个缺失值特征向量，paddding成0
        """
        super(FM, self).__init__()
        self.n_field=n_field             # 原始特征数目
        self.n_features=n_features       # 连续+离散特征的总数目 (离散特征one-hot+unique化后的。且算上最终的一个padding)
        self.k=embed_size
        
        self.W=nn.Embedding(n_features,1,padding_idx=0)                       # 每个特征对应的wi。位置0是0对梯度无贡献。embedding默认是N(0,1)
        self.w0=nn.Parameter(torch.zeros([1,]))                               # b初始化为0
        self.feature_embed=nn.Embedding(n_features,embed_size,padding_idx=0)  # 每个特征对应的embedding
        self.droplayer=nn.Dropout(dropout)
        
        self.weight=nn.Parameter(0.5*torch.ones([2,]))                        # 一阶，二阶score的权重 
                                                                              
        self.__init_weight__()                                                # 初始化权重(可以不调用，用默认的)
            
    def __init_weight__(self):              # 初始化权重.默认是N(0,1)
        inn=n_features-1
        # kaiming_normal_
        nn.init.kaiming_uniform_(self.feature_embed.weight[1:], mode='fan_in', nonlinearity='relu')  # sqrt(6/inn)
        nn.init.normal_(self.W.weight[1:],0, np.sqrt(2.0 /inn))
        
    def forward(self,f_p,f_x): # B,n.  n是每个样本的原始特征数目
        """
        f_p: (B,N)  每个样本的原始特征，根据特征名（连续特征）/特征取值（离散特征）被映射到embedding上的位置。 N:原始特征数目
        f_x: (B,N)  每个样本的原始特征，对应的取值。 离散特征对应的是one-hot后的，所以在对应的f_p上取值为1
        """
        batch_size=f_p.shape[0]
        
        # 一阶score
        w=self.W(f_p).reshape(batch_size,-1)              # B,n   每个样本根据原始特征，找到对应位置处的w
        y_score1=self.w0 + torch.sum(torch.mul(w,f_x),1)  # B,1   wixi+b  B,n -->   B,1.  要是不sum，也可以作为n个特征。之后作为deepFM的改造
        
        # 二阶score
        embed=self.feature_embed(f_p)                     # B,n,d 每个样本根据原始特征，找到对应位置处的embedding
        
        embed=torch.mul(embed,f_x.unsqueeze(2))           # B,n,d 样本的每个向量乘上对应的xi： xi* embedding
                                                          #       tf.mul:按位置乘.广播 (B,n,d) * (B,n,1) ->(B,n,d) 
                                                          #       离散特征对应的xi是1.假设数值型已经归一化
        
        #embed=self.droplayer(embed)
        
        # 每个filed 向量22交互
        e_sum= torch.sum(embed,1)                         # B,d   每个样本。所有embedding对应维度元素相加，得到e_sum
        e_sum_square=torch.square(e_sum)                  # B,d   (e_sum)^2
        
        e_square=torch.square(embed)                      # B,n,d  平方后的
        e_square_sum=torch.sum(e_square,1)                # B,d    每个维度相加
        
        f =0.5*(e_sum_square-e_square_sum)                # B,d    n个embedding，每个维度元素22交互的结果（可作为新特征，拼接到后边）
    
        y_score2= torch.sum(f,1)                          # (B,)
        
        logits=y_score1+y_score2                          # (B,)   最终分数
        
        # TODO:loss加正则项（最后加）/ grad-clip等. 看下train-loop
        
        return logits

### 原始deepFM模型

FM部分相同，但共享底层的embedding部分。把样本原始离散特征和连续特征映射得到的所有embedding，concat,作为后续mlp的输入。输出结果作为deep部分的score,和linear(FM)部分相加

In [110]:
# deepFM： 标准的。 或者上述k个元素作为特征统一拼到deep的
class deepFM(nn.Module):
    def __init__(self,n_field,n_features,embed_size,hiddens,dropout=0,batchnorm=True):
        """
        标准DeepFM
        n_field: 原始特征数目
        n_features: 离散特征one-hot之后,和dense的总特征个数. 这里包含了一个缺失值特征向量，paddding成0
        deep每层：linear + (bn) + relu + (dropout).  输出hidden层。  最后按需单独linear到1
        """
        super(deepFM, self).__init__() 
        self.n_field=n_field             # 原始特征数目
        self.n_features=n_features       # 连续+离散特征的总数目 (离散特征one-hot+unique化后的。且算上最终的一个padding)
        self.k=embed_size
        self.dropout=dropout
        self.batchnorm=batchnorm
        self.hiddens=hiddens             # hiddens:[256,64,32]
    
        # FM-part
        self.W=nn.Embedding(n_features,1,padding_idx=0)                       # 每个特征对应的wi。位置0是0对梯度无贡献。embedding默认是N(0,1)
        self.w0=nn.Parameter(torch.zeros([1,]))                               # b初始化为0
        self.feature_embed=nn.Embedding(n_features,embed_size,padding_idx=0)  # 每个特征对应的embedding
        self.droplayer=nn.Dropout(dropout)            
        
        #self.__init_wide_weight__()                                           # 初始化权重(可以不调用，用默认的)
        
        # deep-part.
        input_size=n_field*embed_size                                         #输入是所有原始特征的embedding拼接   
        self.mlp=self.build_mlp(input_size,hiddens)                           #多层mlp.输出hidden  是一个ModuleList
        self.finallinear=nn.Linear(hiddens[-1],1,bias=True)                   #最后一层linear
        
        #self.__init_deep_weight__()                                           # 初始化权重。(可以按情况调用。现在用kaiming)
            
    def __init_wide_weight__(self):              # 初始化权重.默认是N(0,1)
        inn=n_features-1
        # kaiming_normal_
        nn.init.kaiming_uniform_(self.feature_embed.weight[1:], mode='fan_in', nonlinearity='relu')  # sqrt(6/inn)
        nn.init.normal_(self.W.weight[1:],0, np.sqrt(2.0 /inn))

    def __init_deep_weight__(self):              # 初始化权重
        for layer in self.mlp:
            if (layer.__class__.__name__=='Linear'):  # 每层初始化
                self.init_linear(layer)
        self.init_linear(self.finallinear)
        
    def init_linear(self,layer):
        nn.init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='relu')  #sqrt(2/inn) 或者 sqrt(2/out)
        nn.init.constant_(layer.bias, 0)

    def build_mlp(self,input_size,hiddens):
        """
        hiddens:[256,56,32]
        输出最后一层的hidden节点 （dropout+激活后的），可直接linear+sigmoid到deepscore. 也可以拼其他特征后再linear
        """        
        layers=nn.ModuleList()
        
        hiddens.insert(0,input_size)     #  [inputsize,256,56,32]
        num_layer=len(hiddens)-1         #  3层
        
        for i in range(0,len(hiddens)-1):
            
            # 线性层
            in_dim=hiddens[i]
            out_dim=hiddens[i+1]
            layer=nn.Linear(in_dim,out_dim,bias=True)
            layers.append(layer)
            
            # BN
            if self.batchnorm:
                layers.append(nn.BatchNorm1d(out_dim)) 
            
            # active + dropout
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(self.dropout))
                
        return layers  # 拼成一个前后连接的网络。可以在forward里作为一个模块被整体调用
    
    
    def forward(self,f_p,f_x): # B,n.  n是每个样本的原始特征数目
        """
        f_p: (B,N)  每个样本的原始特征，根据特征名（连续特征）/特征取值（离散特征）被映射到embedding上的位置。 N:原始特征数目
        f_x: (B,N)  每个样本的原始特征，对应的取值。 离散特征对应的是one-hot后的，所以在对应的f_p上取值为1
        output:  FM和deep部分 score相加
        """
        batch_size=f_p.shape[0]
        
        # FM-part    每个filed 向量22交互
        w=self.W(f_p).reshape(batch_size,-1)              # B,n   每个样本根据原始特征，找到对应位置处的w
        y_score1=self.w0 + torch.sum(torch.mul(w,f_x),1)  # B,1   wixi+b  B,n -->   B,1.  要是不sum，也可以作为n个特征。
        
        embed=self.feature_embed(f_p)                     # B,n,d 每个样本根据原始特征，找到对应位置处的embedding 
        embed=torch.mul(embed,f_x.unsqueeze(2))           # B,n,d 样本的每个向量乘上对应的xi： xi* embedding
        e_sum= torch.sum(embed,1)                         # B,d   每个样本。所有embedding对应维度元素相加，得到e_sum
        e_sum_square=torch.square(e_sum)                  # B,d   (e_sum)^2
        e_square=torch.square(embed)                      # B,n,d  平方后的
        e_square_sum=torch.sum(e_square,1)                # B,d    每个维度相加
        f =0.5*(e_sum_square-e_square_sum)                # B,d    n个embedding，每个维度元素22交互的结果（可作为新特征，拼接到后边）
        y_score2= torch.sum(f,1)                          # (B,)
        
        # deep-part
        x=torch.reshape(embed,[-1,self.n_field*self.k])           # 输入是所有原始特征对应embedding的拼接
        for layer in self.mlp:
            if(layer.__class__.__name__=='Linear'):
                print(x.shape,x.mean(),x.std())        # 可以画一下数据分布
            x=layer(x)
        
        y_deep=self.finallinear(x).squeeze(-1)         # (B，)
        
        logits=y_score1+y_score2 +y_deep
        
        return logits

In [111]:
n_field=len(dis_col+con_col)           # 原始特征数目
n_features=len(f_trans.feature_id_map) # 连续+离散特征的总数目 (离散特征one-hot+unique化后的。且算上最终的一个padding)
embed_size=8

In [113]:
hiddens=[256,56,32]
model=deepFM(n_field,n_features,embed_size,hiddens,dropout=0.5)
#model=FM(n_field,n_features,embed_size)

In [115]:
model

deepFM(
  (W): Embedding(319, 1, padding_idx=0)
  (feature_embed): Embedding(319, 8, padding_idx=0)
  (droplayer): Dropout(p=0.5, inplace=False)
  (mlp): ModuleList(
    (0): Linear(in_features=312, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=256, out_features=56, bias=True)
    (5): BatchNorm1d(56, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=56, out_features=32, bias=True)
    (9): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.5, inplace=False)
  )
  (finallinear): Linear(in_features=32, out_features=1, bias=True)
)

In [116]:
logits=model(f_p,f_v)
target=y
loss_func= nn.BCEWithLogitsLoss()    #  mean [yn⋅logσ(xn)+(1−yn)⋅log(1−σ(xn))]
loss=loss_func(logits,target.float())

torch.Size([32, 312]) tensor(-148.8390, grad_fn=<MeanBackward0>) tensor(5103.7476, grad_fn=<StdBackward>)
torch.Size([32, 256]) tensor(0.3615, grad_fn=<MeanBackward0>) tensor(0.9536, grad_fn=<StdBackward>)
torch.Size([32, 56]) tensor(0.3378, grad_fn=<MeanBackward0>) tensor(0.9587, grad_fn=<StdBackward>)
torch.Size([32])
torch.Size([32])
torch.Size([32])


In [100]:
target

tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0])

In [117]:
logits

tensor([-3.0736e+02, -5.1768e+06, -1.3688e+04, -1.3040e+06, -1.8214e+07,
        -3.6906e+07, -1.1229e+07, -1.7264e+05, -5.4649e+05, -4.4986e+02,
        -1.8052e+04,  1.6355e+05, -1.3486e+07, -2.0230e+05,  5.9057e+06,
         1.1068e+04, -3.4805e+05,  4.8048e+04, -1.3879e+06, -4.6276e+05,
         5.4990e+04, -1.5501e+06,  2.0458e+04,  5.3907e+03, -2.3038e+03,
        -7.0575e+05, -2.3822e+07, -1.0117e+07, -5.6195e+05, -8.3120e+04,
        -2.7726e+06, -1.0439e+08], grad_fn=<AddBackward0>)

In [118]:
loss

tensor(1133546.6250, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [119]:
model.feature_embed.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.8664, -1.2823, -0.5834,  ...,  0.3775,  2.6724, -0.1802],
        [ 0.5622,  1.9850,  0.3970,  ..., -0.2929, -1.2681, -0.3383],
        ...,
        [ 0.1856, -1.0168,  0.9001,  ...,  0.6181,  0.2193, -1.1030],
        [ 0.9092,  2.0307, -0.8207,  ...,  1.0724,  0.7585,  1.8692],
        [ 2.3111, -0.8200,  0.4083,  ..., -0.0423,  0.8849, -0.5459]],
       requires_grad=True)

### train_loop